In [ ]:
import gc
import math
import random

import torch
import torch.nn as nn

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from tqdm import trange
from pathlib import Path

In [ ]:
# CONFIG

# Training
LR = 0.001
BS = 32 # DO NOT CHANGE ATM fixed to numpy folder as each bach is saved as a numpy array 

DEVICE = 'cuda'

CHARSET = " .,!abcdefghijklmnopqrstuvwxyz"


 # TODO: Clean this up
FOLDER = Path('data/LJSpeech-1.1/wavs')

path_numpy = FOLDER.parent / Path('numpy')
path_data = path_numpy / Path('data')
path_labels = path_numpy / Path('labels')
path_input = path_numpy / Path('input_lengths')
path_target = path_numpy / Path('target_lengths')

path_models = Path('models')

path_models.mkdir(exist_ok=True)

In [ ]:
# Model (Simple to get started)

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()

        self.ln1 = nn.Sequential(
                                nn.Linear(50,100),
                                nn.ReLU(),                                
                                nn.Linear(100,500),
                                nn.ReLU(),
                                nn.Linear(500,500),
                                nn.ReLU(),
                                )

        self.gru = nn.GRU(500, 500, batch_first=False)
        self.ln2 = nn.Sequential(
                        nn.Linear(500, 150),
                        nn.ReLU(),
                        nn.Linear(150,len(CHARSET)),
                         )

    def forward(self, x):
        x = self.ln1(x)
        x = nn.functional.relu(self.gru(x)[0])
        x = self.ln2(x)
        x = torch.nn.functional.log_softmax(x, dim=2)
        return x

model = Net().to(DEVICE)


In [ ]:
# How are shapes with CTC loss working? 
# INPUT [time,batch,mels]
inn = torch.rand(1114,32,50, device=DEVICE)
print(f"Input shape: {inn.shape}")
test = model(inn)

print(f"Output shape: {test.shape}")
# OUTPUT [time,batch,len(CHARSET)]

In [ ]:
# Criterion (Loss function) and Optimizer
criterion  = nn.CTCLoss(reduction='mean', zero_infinity=True)
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

In [ ]:
# TODO: we need to shuffle, currently every batch and epoch are the same
# TODO: Normalize data?
# TODO: Char mappings how are padding handled vs space 

EPOCHS = 50

traning_loss = []

t = trange(EPOCHS)

for epoch in t:
    for batch in range(408):
        batch = random.randint(0, 408) # BUG: HAX: TODO: BAD
        inputs = np.load(path_data / Path(str(batch) + '.npy'))
        labels = np.load(path_labels / Path(str(batch) + '.npy'))
        input_lengths = np.load(path_input / Path(str(batch) + '.npy'))
        target_lengths = np.load(path_target / Path(str(batch) + '.npy'))

        inputs = torch.tensor(inputs).reshape(-1,BS,50).float().to(DEVICE)
        labels = torch.tensor(labels).float().to(DEVICE)
        input_lengths = torch.tensor(input_lengths)
        target_lengths = torch.tensor(target_lengths)
        
        inputs = torch.nn.functional.normalize(inputs)

        optimizer.zero_grad() # Reset gradient to zero

        # Forward
        output = model(inputs) 

        # Compute loss
        loss = criterion(output, labels, input_lengths,target_lengths)

        # Update loss
        loss.backward()

        # optimizer
        optimizer.step()
        if batch % 10 == 0:
            t.set_description(f"Loss: {loss.item()}")

        del inputs
        del labels
    
    if True:
        torch.save(model.state_dict(), f"models/{str(epoch)}.pt")
    traning_loss.append(loss.item())


In [ ]:
plt.plot(traning_loss)

In [ ]:
# TEST
TEST = 2
TEST2 = 0

data = np.load(path_data / Path(str(TEST) + '.npy'))
data = torch.tensor(data[TEST2]).float()
data = data[:,:,None].permute(1,2,0).to(DEVICE)
print(data.shape)

# Run the model
output = model(data)
output = output[:, 0, :].argmax(dim=1)

estimate = []
for char in output:
    if char != 0:
        estimate.append(CHARSET[char])
print(''.join(estimate))

In [ ]:
# VERIFY
labels = np.load(path_labels / Path(str(batch) + '.npy'))

correct = []
for char in labels[0]:
    correct.append(CHARSET[char])
print(''.join(correct))